# PHASE 4 - STEP 4.2: EFFICIENTNET-B0 TRAINING

**Objective:** Train EfficientNet-B0 with transfer learning for improved accuracy

**Expected Results:**
- Training time: ~2-3 hours (with MPS) or ~4-6 hours (CPU)
- Validation accuracy: ~88-92%
- Significant improvement over baseline (+15-20%)

**Strategy:**
- Phase 1: Freeze base model, train top layers (~15 epochs)
- Phase 2: Unfreeze, fine-tune entire model (~40 epochs)
- Total: ~55 epochs

## 1. IMPORTS & SETUP

In [2]:
# Suppress warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
)

# Print versions
print(f"TensorFlow version: {tf.__version__}")

# Check Keras version (Keras 3 doesn't have __version__)
try:
    print(f"Keras version: {keras.__version__}")
except AttributeError:
    print(f"Keras version: 3.x (integrated with TensorFlow {tf.__version__})")

# Check GPU/Metal availability
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU available: {len(gpus) > 0}")
if len(gpus) > 0:
    print(f"GPU devices: {gpus}")
    print(f"✅ Running on Apple Silicon with Metal acceleration")
else:
    print(f"Running on CPU only")

# Set plot style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
        print("⚠️  Seaborn style not available, using default")

sns.set_palette("husl")

print("\n✅ All imports successful!")


TensorFlow version: 2.15.0
Keras version: 3.x (integrated with TensorFlow 2.15.0)
GPU available: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Running on Apple Silicon with Metal acceleration

✅ All imports successful!


## 2. CONFIGURATION

In [3]:
# Paths
DATA_DIR = Path('../data/processed')
MODEL_DIR = Path('../models')
OUTPUT_DIR = Path('../outputs')

# Create directories
MODEL_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_PHASE1 = 15  # Freeze base, train top
EPOCHS_PHASE2 = 40  # Fine-tune entire model
LEARNING_RATE_PHASE1 = 0.001
LEARNING_RATE_PHASE2 = 0.0001  # 10× lower for fine-tuning

# Class names
CLASS_NAMES = ['battery', 'biological', 'cardboard', 'glass', 
               'metal', 'paper', 'plastic', 'textile', 'trash']
NUM_CLASSES = len(CLASS_NAMES)

print("Configuration:")
print(f"  Image size: {IMG_SIZE}×{IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Phase 1 epochs: {EPOCHS_PHASE1} (freeze base)")
print(f"  Phase 2 epochs: {EPOCHS_PHASE2} (fine-tune)")
print(f"  Total max epochs: {EPOCHS_PHASE1 + EPOCHS_PHASE2}")
print(f"  Classes: {NUM_CLASSES}")
print(f"\n✅ Configuration set!")

Configuration:
  Image size: 224×224
  Batch size: 32
  Phase 1 epochs: 15 (freeze base)
  Phase 2 epochs: 40 (fine-tune)
  Total max epochs: 55
  Classes: 9

✅ Configuration set!


## 3. LOAD CLASS WEIGHTS

In [4]:
# Load class weights
weights_file = DATA_DIR / 'class_weights_simple.json'

with open(weights_file, 'r') as f:
    weights_data = json.load(f)

# Convert to format needed by Keras
class_weights = {int(k): v for k, v in weights_data['recommended_weights'].items()}

print("Class weights loaded:")
for class_idx, weight in class_weights.items():
    print(f"  {CLASS_NAMES[class_idx]:12s}: {weight:.4f}")

print("\n Class weights loaded!")

Class weights loaded:
  battery     : 1.8245
  biological  : 1.7504
  cardboard   : 1.9358
  glass       : 0.8578
  metal       : 2.2416
  paper       : 1.6408
  plastic     : 1.9934
  textile     : 0.2360
  trash       : 2.4764

 Class weights loaded!


## 4. CREATE DATA GENERATORS (STRONGER AUGMENTATION)

In [ ]:
print("Creating data generators...\n")

# Training generator - STRONGER augmentation than baseline
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,           # More rotation
    width_shift_range=0.25,      # More shift
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,          # Add vertical flip
    fill_mode='nearest'
)

# Validation generator - NO augmentation
val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    DATA_DIR / 'train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    DATA_DIR / 'val',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\nTraining steps per epoch: {len(train_generator)}")
print(f"Validation steps per epoch: {len(val_generator)}")
print("\n✅ Data generators created!")

## 5. BUILD EFFICIENTNET-B0 MODEL

In [ ]:
print("Building EfficientNet-B0 model...\n")

# Load pre-trained EfficientNet-B0
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',  # Pre-trained on ImageNet
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling=None
)

print(f"✅ Base model loaded: EfficientNet-B0")
print(f"   Pre-trained on: ImageNet")
print(f"   Base parameters: {base_model.count_params():,}")

# Freeze base model initially (Phase 1)
base_model.trainable = False
print(f"   Status: FROZEN (will train in Phase 2)")

# Build complete model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
], name='EfficientNetB0_WasteClassification')

# Display model summary
print("\n" + "="*70)
print("MODEL ARCHITECTURE")
print("="*70)
model.summary()

print("\n" + "="*70)
print("MODEL STATISTICS")
print("="*70)
print(f"Total parameters:     {model.count_params():,}")
trainable = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable = sum([tf.size(w).numpy() for w in model.non_trainable_weights])
print(f"Trainable (Phase 1):  {trainable:,}")
print(f"Non-trainable:        {non_trainable:,}")
print("="*70)

print("\n✅ Model built successfully!")

## 6. COMPILE MODEL - PHASE 1 (WARMUP)

In [ ]:
print("Compiling model for Phase 1 (Warmup)...\n")

# Use legacy optimizer for Mac M4 compatibility
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE_PHASE1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled!")
print(f"   Phase: 1 (Warmup - Train top layers only)")
print(f"   Optimizer: Adam (legacy, optimized for M4)")
print(f"   Learning rate: {LEARNING_RATE_PHASE1}")
print(f"   Loss: Categorical Crossentropy")
print(f"   Metrics: Accuracy")

## 7. SETUP CALLBACKS - PHASE 1

In [ ]:
print("Setting up callbacks for Phase 1...\n")

# Callbacks for Phase 1
callbacks_phase1 = [
    ModelCheckpoint(
        filepath=str(MODEL_DIR / 'efficientnet_b0_phase1.h5'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    EarlyStopping(
        monitor='val_accuracy',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    ),
    
    CSVLogger(
        str(OUTPUT_DIR / 'efficientnet_phase1_log.csv'),
        append=True
    )
]

print("✅ Callbacks configured:")
print("   - ModelCheckpoint: Save best model")
print("   - EarlyStopping: patience=8")
print("   - ReduceLROnPlateau: patience=4, factor=0.5")
print("   - CSVLogger: Log training metrics")

## 8. TRAIN - PHASE 1 (WARMUP: ~15 EPOCHS, ~20-30 MIN)

**What's happening:**
- Base EfficientNet layers are FROZEN
- Only training the custom top layers (Dense, Dropout, BatchNorm)
- This adapts the output to our 9 waste classes
- Expected time: ~20-30 minutes with MPS

In [ ]:
print("="*70)
print("PHASE 1: WARMUP TRAINING (FREEZE BASE, TRAIN TOP)")
print("="*70)
print(f"Base model: FROZEN")
print(f"Training: Top layers only")
print(f"Max epochs: {EPOCHS_PHASE1}")
print(f"Learning rate: {LEARNING_RATE_PHASE1}")
print(f"\nExpected time: ~20-30 minutes with MPS")
print("="*70)
print("\n🚀 Starting Phase 1 training...\n")

history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_PHASE1,
    class_weight=class_weights,
    callbacks=callbacks_phase1,
    verbose=1
)

print("\n" + "="*70)
print("✅ PHASE 1 COMPLETE!")
print("="*70)
print(f"Best val accuracy: {max(history_phase1.history['val_accuracy']):.4f}")
print("\nModel saved: efficientnet_b0_phase1.h5")
print("\nReady for Phase 2: Fine-tuning entire model...")

## 9. PREPARE FOR PHASE 2 (FINE-TUNING)

In [ ]:
print("="*70)
print("PREPARING PHASE 2: FINE-TUNING")
print("="*70)

# Unfreeze the base model
base_model.trainable = True

print(f"\n✅ Base model UNFROZEN")
print(f"   All {len(base_model.layers)} layers now trainable")

# Count trainable parameters
trainable = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable = sum([tf.size(w).numpy() for w in model.non_trainable_weights])

print(f"\nModel statistics (Phase 2):")
print(f"  Total parameters:     {model.count_params():,}")
print(f"  Trainable:            {trainable:,}")
print(f"  Non-trainable:        {non_trainable:,}")

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE_PHASE2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n✅ Model recompiled for fine-tuning")
print(f"   Learning rate: {LEARNING_RATE_PHASE2} (10× lower)")
print("="*70)

## 10. SETUP CALLBACKS - PHASE 2

In [ ]:
print("Setting up callbacks for Phase 2...\n")

# Callbacks for Phase 2
callbacks_phase2 = [
    ModelCheckpoint(
        filepath=str(MODEL_DIR / 'efficientnet_b0_best.h5'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    EarlyStopping(
        monitor='val_accuracy',
        patience=12,  # More patient for fine-tuning
        restore_best_weights=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        min_lr=1e-8,
        verbose=1
    ),
    
    CSVLogger(
        str(OUTPUT_DIR / 'efficientnet_phase2_log.csv'),
        append=True
    )
]

print("✅ Callbacks configured:")
print("   - ModelCheckpoint: Save best model")
print("   - EarlyStopping: patience=12 (more patient)")
print("   - ReduceLROnPlateau: patience=5, factor=0.5")
print("   - CSVLogger: Log training metrics")

## 11. TRAIN - PHASE 2 (FINE-TUNING: ~40 EPOCHS, ~2-2.5 HOURS)

**What's happening:**
- ALL layers now trainable (base + top)
- Lower learning rate (0.0001) for careful fine-tuning
- This optimizes the entire network for waste classification
- Expected time: ~2-2.5 hours with MPS

**This is where the magic happens! 🎯**

In [ ]:
print("="*70)
print("PHASE 2: FINE-TUNING (TRAIN ENTIRE MODEL)")
print("="*70)
print(f"Base model: UNFROZEN (all layers trainable)")
print(f"Training: Entire model")
print(f"Max epochs: {EPOCHS_PHASE2}")
print(f"Learning rate: {LEARNING_RATE_PHASE2} (10× lower)")
print(f"\nExpected time: ~2-2.5 hours with MPS")
print(f"              ~4-5 hours with CPU")
print("="*70)
print("\n🚀 Starting Phase 2 training...")
print("   ☕ Grab a coffee/tea - this will take a while!\n")

history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_PHASE2,
    class_weight=class_weights,
    callbacks=callbacks_phase2,
    verbose=1
)

print("\n" + "="*70)
print("✅ PHASE 2 COMPLETE!")
print("="*70)
print(f"Best val accuracy: {max(history_phase2.history['val_accuracy']):.4f}")
print("\nModel saved: efficientnet_b0_best.h5")
print("\n🎉 EfficientNet training complete!")

## 12. COMBINE TRAINING HISTORIES

In [ ]:
print("Combining Phase 1 and Phase 2 histories...\n")

# Combine histories
combined_history = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
    'lr': history_phase1.history.get('lr', [LEARNING_RATE_PHASE1] * len(history_phase1.history['loss'])) + 
          history_phase2.history.get('lr', [LEARNING_RATE_PHASE2] * len(history_phase2.history['loss']))
}

# Save combined history
combined_df = pd.DataFrame({
    'epoch': range(len(combined_history['accuracy'])),
    'accuracy': combined_history['accuracy'],
    'val_accuracy': combined_history['val_accuracy'],
    'loss': combined_history['loss'],
    'val_loss': combined_history['val_loss'],
    'lr': combined_history['lr'],
    'phase': ['Phase 1'] * len(history_phase1.history['loss']) + 
             ['Phase 2'] * len(history_phase2.history['loss'])
})

combined_df.to_csv(OUTPUT_DIR / 'efficientnet_training_log.csv', index=False)

print(f"✅ Combined training history saved")
print(f"   Total epochs: {len(combined_history['accuracy'])}")
print(f"   Phase 1: {len(history_phase1.history['loss'])} epochs")
print(f"   Phase 2: {len(history_phase2.history['loss'])} epochs")

## 13. PLOT TRAINING HISTORY

In [ ]:
print("Creating training visualizations...\n")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('EfficientNet-B0 Training History (Two-Phase)', fontsize=16, fontweight='bold')

epochs = range(len(combined_history['accuracy']))
phase1_end = len(history_phase1.history['loss'])

# Plot 1: Accuracy
ax = axes[0, 0]
ax.plot(epochs, combined_history['accuracy'], 'b-', linewidth=2, label='Train')
ax.plot(epochs, combined_history['val_accuracy'], 'r-', linewidth=2, label='Validation')
ax.axvline(x=phase1_end, color='green', linestyle='--', linewidth=2, label='Phase 1→2')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Annotate best
best_val_acc = max(combined_history['val_accuracy'])
best_epoch = combined_history['val_accuracy'].index(best_val_acc)
ax.plot(best_epoch, best_val_acc, 'g*', markersize=15)
ax.annotate(f'Best: {best_val_acc:.4f}', 
            xy=(best_epoch, best_val_acc),
            xytext=(best_epoch + 2, best_val_acc - 0.05),
            arrowprops=dict(arrowstyle='->', color='green', lw=2),
            fontsize=10, fontweight='bold', color='green')

# Plot 2: Loss
ax = axes[0, 1]
ax.plot(epochs, combined_history['loss'], 'b-', linewidth=2, label='Train')
ax.plot(epochs, combined_history['val_loss'], 'r-', linewidth=2, label='Validation')
ax.axvline(x=phase1_end, color='green', linestyle='--', linewidth=2, label='Phase 1→2')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax.set_title('Model Loss', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Learning Rate
ax = axes[1, 0]
ax.plot(epochs, combined_history['lr'], 'orange', linewidth=2)
ax.axvline(x=phase1_end, color='green', linestyle='--', linewidth=2, label='Phase 1→2')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Learning Rate', fontsize=12)
ax.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Summary
ax = axes[1, 1]
ax.axis('off')

final_train_acc = combined_history['accuracy'][-1]
final_val_acc = combined_history['val_accuracy'][-1]

summary_text = f"""
EFFICIENTNET-B0 TRAINING SUMMARY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ARCHITECTURE:
├─ Base: EfficientNet-B0 (ImageNet)
├─ Parameters: ~5.3M
└─ Custom top layers: 3

TRAINING STRATEGY:
├─ Phase 1: Freeze base, train top
│   └─ Epochs: {len(history_phase1.history['loss'])}
├─ Phase 2: Fine-tune entire model
│   └─ Epochs: {len(history_phase2.history['loss'])}
└─ Total epochs: {len(combined_history['accuracy'])}

FINAL RESULTS:
├─ Train Accuracy:   {final_train_acc:.4f} ({final_train_acc*100:.2f}%)
├─ Val Accuracy:     {final_val_acc:.4f} ({final_val_acc*100:.2f}%)
└─ Best Val Acc:     {best_val_acc:.4f} ({best_val_acc*100:.2f}%)

STATUS: ✅ COMPLETE!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

ax.text(0.1, 0.5, summary_text, fontsize=11, verticalalignment='center',
        family='monospace', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.tight_layout()

# Save plot
plot_path = OUTPUT_DIR / 'efficientnet_training_history.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"✅ Plot saved: {plot_path}")

plt.show()

print("\n✅ Visualization complete!")

## 14. EVALUATE MODEL

In [ ]:
print("="*70)
print("FINAL EVALUATION")
print("="*70)

# Load best model
best_model = keras.models.load_model(MODEL_DIR / 'efficientnet_b0_best.h5')
print("✅ Best model loaded\n")

# Evaluate on validation set
print("Evaluating on validation set...")
val_loss, val_accuracy = best_model.evaluate(val_generator, verbose=1)

print("\n" + "="*70)
print("RESULTS")
print("="*70)
print(f"Validation Loss:      {val_loss:.4f}")
print(f"Validation Accuracy:  {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print("="*70)

# Save results
results = {
    'model': 'EfficientNet-B0',
    'timestamp': datetime.now().isoformat(),
    'phase1_epochs': len(history_phase1.history['loss']),
    'phase2_epochs': len(history_phase2.history['loss']),
    'total_epochs': len(combined_history['accuracy']),
    'best_val_accuracy': float(best_val_acc),
    'best_epoch': int(best_epoch),
    'final_val_loss': float(val_loss),
    'final_val_accuracy': float(val_accuracy),
    'parameters': int(model.count_params())
}

with open(OUTPUT_DIR / 'efficientnet_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✅ Results saved: efficientnet_results.json")

## 15. COMPARE WITH BASELINE

In [ ]:
print("="*70)
print("MODEL COMPARISON: BASELINE vs EFFICIENTNET")
print("="*70)

# Load baseline results
baseline_log = pd.read_csv(OUTPUT_DIR / 'baseline_training_log.csv')
baseline_best = baseline_log['val_accuracy'].max()

# EfficientNet results
efficient_best = best_val_acc

# Calculate improvement
improvement = efficient_best - baseline_best
improvement_pct = (improvement / baseline_best) * 100

print("\nRESULTS:")
print(f"  Baseline CNN:       {baseline_best:.4f} ({baseline_best*100:.2f}%)")
print(f"  EfficientNet-B0:    {efficient_best:.4f} ({efficient_best*100:.2f}%)")
print(f"\nIMPROVEMENT:")
print(f"  Absolute:           +{improvement:.4f} (+{improvement*100:.2f}%)")
print(f"  Relative:           +{improvement_pct:.2f}% improvement")

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

models = ['Baseline CNN', 'EfficientNet-B0']
accuracies = [baseline_best * 100, efficient_best * 100]
colors = ['#3498db', '#2ecc71']

bars = ax.bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.2f}%',
            ha='center', va='bottom', fontsize=14, fontweight='bold')

# Add improvement arrow
ax.annotate('', xy=(1, efficient_best * 100), xytext=(0, baseline_best * 100),
            arrowprops=dict(arrowstyle='->', lw=3, color='red'))
ax.text(0.5, (baseline_best * 100 + efficient_best * 100) / 2,
        f'+{improvement*100:.2f}%\nimprovement',
        ha='center', va='center', fontsize=12, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

ax.set_ylabel('Validation Accuracy (%)', fontsize=12, fontweight='bold')
ax.set_title('Model Comparison: Baseline vs EfficientNet-B0', fontsize=14, fontweight='bold')
ax.set_ylim([0, 100])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Comparison plot saved: model_comparison.png")
print("="*70)

# Summary
print("\n" + "="*70)
print("🎉 PHASE 4.2 COMPLETE!")
print("="*70)
print(f"\n✅ EfficientNet-B0 achieved {efficient_best*100:.2f}% validation accuracy")
print(f"✅ {improvement_pct:.1f}% improvement over baseline")
print(f"\nFiles saved:")
print(f"  - models/efficientnet_b0_best.h5")
print(f"  - outputs/efficientnet_training_log.csv")
print(f"  - outputs/efficientnet_training_history.png")
print(f"  - outputs/efficientnet_results.json")
print(f"  - outputs/model_comparison.png")
print("\n🚀 Ready for Phase 4.3: Model Comparison & Analysis!")
print("="*70)